In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 128
#block_size = 64
max_iters = 10
learning_rate = 3e-3
eval_iters = 2
eval_interval=500
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

print(device)

cuda


In [2]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Set up your Reddit API credentials
import praw
import json
reddit = praw.Reddit(
    client_id='_aAnJg3YfIZqHteF_Gx4vg',
    client_secret='O6kpd18vcDpmo7JcZEPhyXGjgEIgUA',
    user_agent='My_Bot/0.1 by u/Fuzzy_Bookkeeper_118'
)

# Specify the URL of the Reddit post
url = 'https://www.reddit.com/r/SaaS/comments/1ho8qvx/ai_agents_are_they_the_future_or_just_hype/'

# Fetch the submission
submission = reddit.submission(url=url)

# Extract comments
comments = []
submission.comments.replace_more(limit=None)
for comment in submission.comments.list():
    comments.append(comment.body)

# Save comments to a JSON file
with open('comments.json', 'w') as f:
    json.dump(comments, f, indent=4)

print("Comments extracted and saved to comments.json")

Comments extracted and saved to comments.json


In [5]:
# Load the extracted comments
with open('comments.json', 'r') as f:
    comments = json.load(f)

# Create a single string of text
text = ' '.join(comments)

# Create the vocabulary and encoding functions
chars=sorted(list(set(text)))
vocab_size=len(chars)  
print(vocab_size)

74


In [6]:
import tempfile
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda l:' '.join([int_to_string[i] for i in l])

# Save vocabulary to vocab.txt
with open('vocab.txt', 'w') as f:
    for ch in chars:
        f.write(f"{ch}\t{string_to_int[ch]}\n")
print("Vocabulary saved to vocab.txt")

# Encode the text data
encoded_data = encode(text)
# Use a temporary directory
with tempfile.TemporaryDirectory() as tmpdir:
    encoded_data_file = os.path.join(tmpdir, 'encoded_data.bin')

    # Save encoded data to a binary file using memory mapping
    try:
        with open(encoded_data_file, 'wb') as f:
            f.write(bytearray(encoded_data))
        print("Encoded data saved to encoded_data.bin")
    except OSError as e:
        print(f"Error saving encoded data: {e}")

    # Memory map the binary file
    try:
        with open(encoded_data_file, 'r+b') as f:
            data = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
        print("Memory mapping successful")

        # Convert the memory-mapped data to a tensor
        try:
            data_tensor = torch.tensor(list(data), dtype=torch.long)
            print("Dataset loaded")  # After loading dataset
        except Exception as e:
            print(f"Error converting to tensor: {e}")
        finally:
            data.close()  # Close the memory-mapped file
    except OSError as e:
        print(f"Error with memory mapping: {e}")

print("Temporary directory cleaned up")


Vocabulary saved to vocab.txt
Encoded data saved to encoded_data.bin
Memory mapping successful
Dataset loaded
Temporary directory cleaned up


In [7]:
  # Calculate data length and max block size
data_len = len(data_tensor)  # Example: data_len = 10000
max_block_size = data_len // batch_size  # Example: max_block_size = 78
            
# Set block size to a suitable value
block_size = min(64, max_block_size)  # Use 64 or max_block_size, whichever is smaller

n = int(0.8 * len(data_tensor))
train_data = data_tensor[:n]
val_data = data_tensor[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    if len(data) < block_size * batch_size:
        raise ValueError("Data size is smaller than the required block size.")
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    x = x.squeeze(-1)  # Remove the last dimension if it is 1
    y = y.squeeze(-1)  # Remove the last dimension if it is 1
    print(f"x shape: {x.shape}, y shape: {y.shape}")  # Debugging print statement
    return x, y

In [8]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x


         
#Normalization: Takes place in the __init__ method with self.ln_f = nn.LayerNorm(n_embd), and is applied at the end of the blocks in the transformer architecture.
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, n_embd) #Maps each token in the vocabulary to a dense vector of size n_embd.
        self.position_embedding_table= nn.Embedding(block_size, n_embd) #Maps each position in the input sequence to a dense vector of size n_embd
        self.blocks=nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) #A sequence of transformer blocks (Block),
        #each consisting of multi-head self-attention and feed-forward layers

        self.ln_f=nn.LayerNorm(n_embd) #final layer norm,Applies layer normalization to stabilize and speed up training.
        self.lm_head=nn.Linear(n_embd, vocab_size) #language modelling head, doing the final linear transformation, 
        #A linear layer that maps the final hidden states to the vocabulary size, used to predict the next token.

        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets=None):  #Computes the output of the model given an input sequence of token indices. If targets are provided, it also computes the loss.
        #logits=self.token_embedding_table(index)  
        print(f"index shape: {index.shape}")  # Debugging print statement
        B, T=index.shape
        
        #Converts the input token indices (index) into embedding vectors using the token embedding table

        #idx and targets are both (B,T) tensor of integers
        tok_emb =self.token_embedding_table(index)  #Embeds the token indices (idx), converting them into corresponding embedding vectors.
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C) Generates positional encodings for each position in the sequence, ensuring the model can understand the order of tokens.
        x = tok_emb + pos_emb #(B,T,C)Adds the token embeddings and positional embeddings element-wise to create a combined representation of tokens with their respective positions.
        x = self.blocks(x) #(B, T, C)Passes the combined embeddings through the transformer blocks, 
        #applying attention mechanisms and other transformations to capture relationships between tokens
        x = self.ln_f(x) #(B,T,C)Applies layer normalization to the transformed embeddings, helping stabilize and improve the training process.
        logits = self.lm_head(x) #(B,T, vocab_size)
        #Passes the normalized embeddings through the final layer (language model head) to produce the logits, which represent the model's raw output before applying any activation functions.
        if targets is None:
            loss=None
        else:
            B, T, C=logits.shape#If targets are provided, reshapes the logits and targets tensors, and computes the cross-entropy loss.
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)

        return logits,loss
#Decoding: The generate method is responsible for decoding, as it generates new token indices based on the model's predictions.
    def generate(self, index, max_new_tokens):  #Generates a sequence of tokens given an initial input context by sampling from the model's predictions.
        #index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss=self.forward(index)
            #focus only on the last time step
            logits=logits[:, -1, :] #(B,C)   # focus only on the last time step 
            #apply softmax to get probabilities
            #Softmax: Implemented in the generate method using F.softmax(logits, dim=-1) to convert logits to probabilities.
            probs=F.softmax(logits, dim=-1)  # apply softmax to get probabilities, converts logits to probabilities
            #sample from distribution
            index_next=torch.multinomial(probs, num_samples=1) #(B,1) )  # sample the next token index from probability distribution
            #append sampled index to the running sequence
            index= torch.cat((index, index_next), dim=1) #(B, T+1) # append sampled index to the running sequence
        return index

model=GPTLanguageModel(vocab_size)
print("Model initialized")  # After initializing the model
m=model.to(device)

Model initialized


In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(f"Iteration: {iter}")  # Debugging print statement
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    
    print(f"Loss: {loss.item()}")  # Debugging print statement
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

Iteration: 0


ValueError: Data size is smaller than the required block size.

In [11]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

index shape: torch.Size([1, 22])
index shape: torch.Size([1, 23])
index shape: torch.Size([1, 24])
index shape: torch.Size([1, 25])
index shape: torch.Size([1, 26])
index shape: torch.Size([1, 27])
index shape: torch.Size([1, 28])
index shape: torch.Size([1, 29])
index shape: torch.Size([1, 30])
index shape: torch.Size([1, 31])
index shape: torch.Size([1, 32])
index shape: torch.Size([1, 33])
index shape: torch.Size([1, 34])
index shape: torch.Size([1, 35])
index shape: torch.Size([1, 36])
index shape: torch.Size([1, 37])
index shape: torch.Size([1, 38])
index shape: torch.Size([1, 39])
index shape: torch.Size([1, 40])
index shape: torch.Size([1, 41])
index shape: torch.Size([1, 42])
index shape: torch.Size([1, 43])
index shape: torch.Size([1, 44])
index shape: torch.Size([1, 45])


RuntimeError: The size of tensor a (44) must match the size of tensor b (45) at non-singleton dimension 2